In [1]:

from matplotlib import pyplot
#https://www.seligson.fi/graafit/eurooppa_exc.csv
#https://www.seligson.fi/graafit/aasia_exc.csv
#https://www.seligson.fi/graafit/suomi_exc.csv
#calls_df.to_csv("calls.csv", index=False)

#calls_df, = pd.read_html("http://apps.sandiego.gov/sdfiredispatch/")
'''pamerikka = pd.read_csv("https://www.seligson.fi/graafit/pohjoisamerikka_exc.csv",sep=";",names=("Date","Value"))
aasia = pd.read_csv("https://www.seligson.fi/graafit/aasia_exc.csv",sep=";",names=("Date","Value"))
eurooppa = pd.read_csv("https://www.seligson.fi/graafit/eurooppa_exc.csv",sep=";",names=("Date","Value"))
suomi = pd.read_csv("https://www.seligson.fi/graafit/suomi_exc.csv",sep=";",names=("Date","Value"))
print(pamerikka[0:5], "\n")
print(aasia[0:5], "\n")
print(eurooppa[0:5], "\n")
print(suomi[0:5], "\n")'''
import pandas as pd
def loadSuomi():
    suomi = pd.read_csv("https://www.seligson.fi/graafit/suomi_exc.csv",sep=";",names=("Date","Value"))
    data=suomi["Value"]
    data=data.str.replace(',','.').astype(float)
    return data
    #pyplot.plot(data)
    #pyplot.show()


In [2]:
from pandas.plotting import autocorrelation_plot
#autocorrelation_plot(data[0:100])
autocorrelation_plot(data)

NameError: name 'data' is not defined

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from pandas import DataFrame
from matplotlib import pyplot

model = ARIMA(data, order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())

residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe())

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from pandas import DataFrame
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error

size = int(len(data) * 0.66)
train, test = data[0:size], data[size:len(data)]
history = [x for x in train]
predictions = list()

for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[3413+t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

error = mean_squared_error(test, predictions)
print(model_fit.summary())

print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

print(residuals.describe())

In [ ]:
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
 
series = data
X = series.values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
pyplot.plot(test[-100:-1])
pyplot.plot(predictions[-100:-1], color='red')
pyplot.show()

In [ ]:
from multiprocessing import Pool


In [ ]:
import warnings
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(disp=0)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	error = mean_squared_error(test, predictions)
	return error

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_cfg = mse, order
					print('ARIMA%s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s MSE=%.3f' % (best_cfg, best_score))

# load dataset
series = loadSuomi()[-200:-1]
# evaluate parameters
p_values = [1, 2, 3, 5, 12]
d_values = [0, 1, 2, 3, 5, 12]
q_values = [0, 1, 2, 3, 5, 12]
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)
'''
ARIMA(0, 0, 0) MSE=8.268
ARIMA(0, 0, 1) MSE=2.134
ARIMA(0, 1, 0) MSE=0.004
ARIMA(0, 1, 1) MSE=0.004
ARIMA(0, 1, 2) MSE=0.004
ARIMA(0, 1, 10) MSE=0.004

-200
ARIMA(1, 0, 0) MSE=0.006
ARIMA(1, 0, 1) MSE=0.006
ARIMA(1, 0, 2) MSE=0.006
ARIMA(1, 0, 3) MSE=0.006
ARIMA(1, 0, 5) MSE=0.006
ARIMA(1, 1, 0) MSE=0.007
ARIMA(1, 2, 0) MSE=0.008
'''

ARIMA(1, 0, 0) MSE=0.006
ARIMA(1, 0, 1) MSE=0.006
ARIMA(1, 0, 2) MSE=0.006
ARIMA(1, 0, 3) MSE=0.006
ARIMA(1, 0, 5) MSE=0.006
ARIMA(1, 1, 0) MSE=0.007
ARIMA(1, 2, 0) MSE=0.008


In [ ]:
import warnings
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from multiprocessing import Pool

def loadSuomi():
    suomi = read_csv("https://www.seligson.fi/graafit/suomi_exc.csv",sep=";",names=("Date","Value"))
    data=suomi["Value"]
    data=data.str.replace(',','.').astype(float)
    return data

# evaluate an ARIMA model for a given order (p,d,q)
# X=data.astype('float32')

def evaluate_arima_model_parallel(arima_order):
    # prepare training dataset
    print("Loading data")
    X=loadSuomi().astype('float32')[-200:-1]
    print("Running with order ", arima_order)
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    error = mean_squared_error(test, predictions)
    print(arima_order,error)

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models_parallel(p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None
    order=[]
    print("Preparing")
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order.append((p,d,q))
    print("Starting")
    p = Pool(4)
    p.map(evaluate_arima_model_parallel, order)
    p.start()
    print("Functions finished")
# load dataset
#series = data
# evaluate parameters
p_values = [1, 2, 3, 5, 12]
d_values = [0, 1, 2, 3, 5, 12]
q_values = [0, 1, 2, 3, 5, 12]
warnings.filterwarnings("ignore")
print("Starting")
evaluate_models_parallel(p_values, d_values, q_values)


Starting
Preparing
Starting
